In [4]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import torch
from torch import nn
import torchlibrosa
import numpy as np
import soundfile
import os
import coremltools as ct
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import librosa
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

def to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)


Using device: cpu



In [5]:
class WaveToSpectrogram(nn.Module):

    def __init__( self, n_fft, hop_length, center=False ):
        super(WaveToSpectrogram, self).__init__()

        self.spec_extractor = torchlibrosa.stft.Spectrogram(
            n_fft=n_fft,
            hop_length=hop_length,
            center=center,
        )

    def forward( self, x ):
        return self.spec_extractor( x )
    

In [4]:
model = WaveToSpectrogram( n_fft=1024, hop_length=512 )
torch.save(model, 'Hello.pt')

waveform, samplerate = soundfile.read( 'bonjour.wav' )
sample_input = waveform[:32000].astype( dtype=np.float32 )

# instantiate model and export it
model = WaveToSpectrogram( n_fft=1024, hop_length=512 )

/Users/hugo/opt/anaconda3/envs/audio/lib/python3.8/site-packages/torchlibrosa/stft.py:193: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = librosa.util.pad_center(fft_window, n_fft)
/Users/hugo/opt/anaconda3/envs/audio/lib/python3.8/site-packages/torch/serialization.py:359: UserWarning: Couldn't retrieve source code for container of type WaveToSpectrogram. It won't be checked for correctness upon loading.
  warnings.warn("Couldn't retrieve source code for container of "


## Convert to Onnx

In [7]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    return torch.device("cpu")
device=get_default_device()

In [ ]:
model_path = 'history/Cnn10_lr_0.005_ep_100_bs_32_valoss_1.5_acc_0.94_recall_0.89_precision_0.89'
#model_path = 'history/MobileNetV1_lr_0.005_ep_100_bs_32_valoss_1.5773910284042358_acc_0.82_recall_0.82_precision_0.82'
waveform, samplerate = soundfile.read( '../urbansound8k/audio/fold1/101415-3-0-2.wav' )
sample_input = waveform[:192000].astype( dtype=np.float32).reshape((1, 192000))

def export_torch_onnx(model, sample_input, filename_onnx):
    torch.onnx.export(
            model,
            torch.from_numpy( sample_input).to(device),
            filename_onnx,
            verbose = True,
            )
model = torch.load(os.path.join(model_path, 'model.pt'))
export_torch_onnx(model, sample_input, os.path.join(model_path, 'model_test.onnx'))


## ONNX to Coreml

In [ ]:
mlmodel = onnx_coreml.convert(model = 'wave_to_spectrogram_model.onnx')

In [ ]:

mlmodel = onnx_coreml.convert(
    model = 'wave_to_spectrogram_model.onnx',
    minimum_ios_deployment_target='13',
)
mlmodel.save(os.path.join(model_path, 'model.mlmodel'))

In [ ]:
import onnx_coreml
import coremltools
model_path = 'history/Cnn10_lr_0.005_ep_100_bs_32_valoss_1.5_acc_0.94_recall_0.89_precision_0.89'

mlmodel = onnx_coreml.convert(
    model = os.path.join(model_path, 'model.onnx'),
    mode = 'classifier',
    minimum_ios_deployment_target='13',
    class_labels='labels.txt',
)
mlmodel.save(os.path.join(model_path, 'model.mlmodel'))

### Pytorch to CoreML

In [25]:

def convert_pytorch_to_coreml(model, example_input: torch.Tensor, output_filename='model.mlpackage') -> None:
    model.eval()
    # Trace the model with exemple_input.
    traced_model = torch.jit.trace(model, example_input, strict=False)
    out = traced_model(example_input)
    # Using image_input in the inputs parameter:
    # Convert to Core ML program using the Unified Conversion API.
    labels = ["air_conditioner",
            "car_horn",
            "children_playing",
            "dog_bark",
            "drilling",
            "engine_idling",
            "gun_shot",
            "jackhammer",
            "siren",
            "street_music"]
    classifier_config_ = ct.ClassifierConfig(labels)

    traced_model.eval()
    model = ct.convert(
        traced_model,
        convert_to="mlprogram",
        classifier_config=classifier_config_,
        inputs=[ct.TensorType(shape=example_input.shape)]
    )
    model.save(output_filename)

### Mobiletnetv2

In [ ]:
model_path = 'history/Cnn10_lr_0.005_ep_100_bs_32_valoss_1.5_acc_0.94_recall_0.89_precision_0.89'
#model_path = 'history/MobileNetV1_lr_0.005_ep_100_bs_32_valoss_1.5773910284042358_acc_0.82_recall_0.82_precision_0.82'
waveform, samplerate = soundfile.read( '../urbansound8k/audio/fold1/101415-3-0-2.wav' )
sample_input = waveform[:192000].astype( dtype=np.float32).reshape((1, 192000))

model = torch.load(os.path.join(model_path, 'model.pt'))



In [2]:
# Load a pre-trained version of MobileNetV2
torch_model = torchvision.models.mobilenet_v2(pretrained=True)
# Set the model in evaluation mode.
torch_model.eval()

# Trace the model with random data.
example_input = torch.rand(1, 3, 224, 224) 
traced_model = torch.jit.trace(torch_model, example_input)
out = traced_model(example_input)

# Using image_input in the inputs parameter:
# Convert to Core ML program using the Unified Conversion API.

traced_model.eval()
model = ct.convert(
    traced_model,
    convert_to="mlprogram",
    inputs=[ct.TensorType(shape=example_input.shape)]
 )

Running MIL Common passes:   0%|          | 0/33 [00:00<?, ? passes/s]/Users/hugo/opt/anaconda3/envs/coremltools3.8/lib/python3.8/site-packages/coremltools/converters/mil/mil/passes/name_sanitization_utils.py:101: UserWarning: Input, 'x.1', of the source model, has been renamed to 'x_1' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Running MIL Clean up passes: 100%|██████████| 8/8 [00:00<00:00,  9.42 passes/s]


In [7]:
# Define a simple layer module we'll reuse in our network.
class Layer(nn.Module):
    def __init__(self, dims):
        super(Layer, self).__init__()
        self.conv1 = nn.Conv2d(*dims)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, (2, 2))
        return x

# A simple network consisting of several base layers.
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.layer1 = Layer((3, 6, 3))
        self.layer2 = Layer((6, 16, 1))

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        return x
model = SimpleNet()  # Instantiate the network.
example = torch.rand(1, 3, 224, 224)  # Example input, needed by jit tracer.
traced = torch.jit.trace(model, example)  # Generate TorchScript by tracing.
scripted = torch.jit.script(model)


## Install Pytorch with coreml 
* python3.8
* pip3 install --pre torch torchvision torchaudio -f https://download.pytorch.org/whl/nightly/cpu/torch_nightly.html
* pip3 install coremltools==5.0b5 protobuf==3.20.1
* pip3 install "numpy<1.24"


#### Mobilenetv2

In [1]:
import torch
import torchvision

from torch.backends._coreml.preprocess import (
    CompileSpec,
    TensorSpec,
    CoreMLComputeUnit,
)

def mobilenetv2_spec():
    return {
        "forward": CompileSpec(
            inputs=(
                TensorSpec(
                    shape=[1, 3, 224, 224],
                ),
            ),
            outputs=(
                TensorSpec(
                    shape=[1, 1000],
                ),
            ),
            backend=CoreMLComputeUnit.ALL,
            allow_low_precision=True,
        ),
    }


model = torchvision.models.mobilenet_v2(pretrained=True)
model.eval()
example = torch.rand(1, 3, 224, 224)
model = torch.jit.trace(model, example)
compile_spec = mobilenetv2_spec()
mlmodel = torch._C._jit_to_backend("coreml", model, compile_spec)
mlmodel._save_for_lite_interpreter("./mobilenetv2_coreml.ptl")

/Users/hugo/opt/anaconda3/envs/coremltools3.8/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugo/opt/anaconda3/envs/coremltools3.8/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.
Running MIL Common passes:   0%|          | 0/33 [00:00<?, ? passes/s]/Users/hugo/opt/anaconda3/envs/coremltools3.8/lib/python3.8/site-packages/coremltools/converters/mil/mil/passes/name_sanitization_utils.py:129: UserWarnin

input {
  name: "input_0"
  type {
    multiArrayType {
      shape: 1
      shape: 3
      shape: 224
      shape: 224
      dataType: FLOAT32
    }
  }
}
output {
  name: "var_647"
  type {
    multiArrayType {
      dataType: FLOAT32
    }
  }
}
metadata {
  userDefined {
    key: "com.github.apple.coremltools.source"
    value: "torch==2.0.0.dev20230104"
  }
  userDefined {
    key: "com.github.apple.coremltools.version"
    value: "5.0b5"
  }
}



### Cnn10

In [21]:
model = torch.load("history/Cnn10_lr_0.005_ep_100_bs_64_valoss_1.49_acc_0.95_recall_0.96_precision_0.96/model.pt", map_location=torch.device('cpu'))


Cnn10(
  (spectrogram_extractor): Spectrogram(
    (stft): STFT(
      (conv_real): Conv1d(1, 513, kernel_size=(1024,), stride=(320,), bias=False)
      (conv_imag): Conv1d(1, 513, kernel_size=(1024,), stride=(320,), bias=False)
    )
  )
  (logmel_extractor): LogmelFilterBank()
  (spec_augmenter): SpecAugmentation(
    (time_dropper): DropStripes()
    (freq_dropper): DropStripes()
  )
  (bn0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_block1): ConvBlock(
    (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_block2): ConvBlock(
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (c

In [18]:
%load_ext autoreload
%autoreload 2
from models import MobileNetV1, Cnn10, MobileNetV2

model_args = {
      'sample_rate': 22050,
      'window_size': 1024,
      'hop_size': 320,
      'mel_bins': 64,
      'fmin': 50,
      'fmax': 20000,
      'classes_num': 10
    }

model_cnn10 = Cnn10(**model_args)
model_cnn10.load_state_dict(torch.load('history/Cnn10_lr_0.005_ep_100_bs_64_valoss_1.49_acc_0.95_recall_0.96_precision_0.96/model_state.pt', map_location=torch.device('cpu')))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/Users/hugo/opt/anaconda3/envs/coremltools3.8/lib/python3.8/site-packages/torchlibrosa/stft.py:193: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = librosa.util.pad_center(fft_window, n_fft)
/Users/hugo/opt/anaconda3/envs/coremltools3.8/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)


<All keys matched successfully>

## avec coremltools 6.1 on arrive à 203/204
enleve le disque stp

In [26]:
model = torch.load("history/Cnn10_lr_0.005_ep_100_bs_64_valoss_1.49_acc_0.95_recall_0.96_precision_0.96/model.pt", map_location=torch.device('cpu'))
dat1, sampling_rate1 = librosa.load('../urbansound8k/audio/fold5/100032-3-0-0.wav')
shape_dat1 = dat1.reshape(1, dat1.shape[0])
input_sample = torch.Tensor(shape_dat1)

convert_pytorch_to_coreml(model_cnn10, input_sample, output_filename='cnn10_96acc.mlpackage')


Running MIL Common passes:   0%|          | 0/39 [00:00<?, ? passes/s]/Users/hugo/opt/anaconda3/envs/coremltools3.8/lib/python3.8/site-packages/coremltools/converters/mil/mil/passes/name_sanitization_utils.py:107: UserWarning: Input, 'input.1', of the source model, has been renamed to 'input_1' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
/Users/hugo/opt/anaconda3/envs/coremltools3.8/lib/python3.8/site-packages/coremltools/converters/mil/mil/passes/name_sanitization_utils.py:135: UserWarning: Output, '283', of the source model, has been renamed to 'var_283' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Running MIL Clean up passes: 100%|██████████| 11/11 [00:00<00:00, 76.81 passes/s]
